<a href="https://colab.research.google.com/github/hyunaeee/upstage/blob/main/claude_fewshot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0. 필요한 디렉토리 생성
from pathlib import Path
Path("./logs").mkdir(exist_ok=True)
Path("./output").mkdir(exist_ok=True)

In [ ]:
pip install anthropic

In [ ]:
# 1. 필요한 라이브러리 임포트
import pandas as pd
from anthropic import Anthropic  # OpenAI 대신 Anthropic 임포트
from rouge_score import rouge_scorer
from tqdm import tqdm
import logging
import json
from datetime import datetime

In [ ]:
# 2. 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f"./logs/summarizer_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [ ]:
# 3. Claude API 설정
anthropic = Anthropic(api_key='...')  # 여기에 Claude API 키를 넣으세요

In [ ]:
# 4. 시스템 프롬프트 정의
SYSTEM_PROMPT = """






"""

In [ ]:
# 5. Few-shot 예시 정의
FEW_SHOT_EXAMPLES = """




"""

In [ ]:
# 5. 데이터 로딩 함수
def load_data(file_path):
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        logger.error(f"Error loading data from {file_path}: {e}")
        raise

# 데이터 로드
train_data = load_data('../data/train.csv')
dev_data = load_data('../data/dev.csv')
test_data = load_data('../data/test.csv')

In [ ]:
# 6. 데이터 전처리: 필요한 컬럼만 선택
train_dialogues = train_data['dialogue'].tolist()
train_summaries = train_data['summary'].tolist()
dev_dialogues = dev_data['dialogue'].tolist()
dev_summaries = dev_data['summary'].tolist()
test_dialogues = test_data['dialogue'].tolist()

In [ ]:
# Claude API를 사용하여 요약 생성
def generate_summary_with_claude(dialogue, index=None):
    try:
        if index is not None:
            logger.info(f"Processing index: {index}")

        response = anthropic.messages.create(
            model="claude-3-opus-20240229",
            system=SYSTEM_PROMPT,
            messages=[
                {
                    "role": "user",
                    "content": f"다음은 대화 요약의 예시입니다:\n{FEW_SHOT_EXAMPLES}\n\n"
                              f"이제 다음 대화를 위 예시들처럼 요약해주세요:\n{dialogue}"
                }
            ],
            max_tokens=150,
            temperature=0.5
        )
        summary = response.content[0].text
        logger.info(f"Summary for index {index}: {summary}")
        return summary
    except Exception as e:
        logger.error(f"Error generating summary: {e}")
        return ""

In [ ]:
# 8. ROUGE 점수 계산 [이전과 동일]
def calculate_rouge_score(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_results = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for pred, ref in zip(predictions, references):
        if pred and ref:
            scores = scorer.score(ref, pred)
            for metric in rouge_results:
                rouge_results[metric].append(scores[metric].fmeasure)

    rouge_avg = {key: sum(value) / len(value) if value else 0
                for key, value in rouge_results.items()}
    return rouge_avg

In [ ]:
# 9. 개발 데이터 평가
def evaluate_on_dev():
    logger.info("Starting evaluation on dev set...")
    predictions = []

    for idx, dialogue in enumerate(tqdm(dev_dialogues, desc="Evaluating on dev set")):
        summary = generate_summary_with_claude(dialogue, idx)  # 함수명 변경
        predictions.append(summary)

    rouge_scores = calculate_rouge_score(predictions, dev_summaries)
    logger.info(f"ROUGE Scores on Dev Set: {rouge_scores}")
    return rouge_scores, predictions


In [ ]:
# 10. 테스트 데이터 요약 생성
def generate_summaries_for_test():
    logger.info("Generating summaries for test set...")
    test_summaries = []

    for idx, dialogue in enumerate(tqdm(test_dialogues, desc="Generating summaries for test set")):
        summary = generate_summary_with_claude(dialogue, idx)  # 함수명 변경
        test_summaries.append(summary)

    return test_summaries

In [ ]:
# 메인 실행 부분 [이전과 동일]
if __name__ == "__main__":
    try:
        # 개발 데이터 평가
        dev_scores, dev_predictions = evaluate_on_dev()

        # 결과 저장
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        # 개발 셋 결과 저장
        dev_results_df = pd.DataFrame({
            'dialogue': dev_dialogues,
            'reference_summary': dev_summaries,
            'generated_summary': dev_predictions
        })

        # 테스트 데이터 요약 생성
        test_summaries = generate_summaries_for_test()

        # 테스트 결과 저장
        test_results_df = pd.DataFrame({
            'fname': test_data['fname'],
            'dialogue': test_data['dialogue'],
            'summary': test_summaries
        })

        dev_results_df.to_csv(f'./output/dev_results_{timestamp}.csv', index=False)
        test_results_df.to_csv(f'./output/test_results_{timestamp}.csv', index=False)

        # 점수 저장
        with open(f'./output/dev_scores_{timestamp}.json', 'w', encoding='utf-8') as f:
            json.dump(dev_scores, f, ensure_ascii=False, indent=2)

        logger.info("All results have been saved to output directory")

    except Exception as e:
        logger.error(f"Pipeline failed: {e}")
        raise

In [ ]:
if __name__ == "__main__":
    main()